In [1]:
import torch
import numpy as np
from torchessian import hessian_matmul
from torchvision.models import resnet18
from itertools import product

In [8]:
n = 2
model = torch.nn.Linear(n, 1)
# torch.manual_seed(2019)
v = torch.rand(sum(p.data.numel() for p in model.parameters()))
x = torch.rand(10, n)
y = x.sum(1)
def loss_function(y_hat, y):
    return ((y_hat.view(-1) - y.view(-1)) * (y_hat.view(-1) - y.view(-1))).sum() / y.numel()
batch = x, y
m = 3
# Exact Hessian

h11 = 2 * (x[:, 0] * x[:, 0]).sum() / x.size(0)
h21 = h12 = 2 * (x[:, 0] * x[:, 1]).sum() / x.size(0)
h31 = h13 = 2 * x[:, 0].sum() / x.size(0)

h22 = 2 * (x[:, 1] * x[:, 1]).sum() / x.size(0)
h32 = h23 = 2 * x[:, 1].sum() / x.size(0)

h33 = 2

H = torch.zeros(3, 3)

for i, j in product(range(3), range(3)):
    exec("H[%d, %d] = h%d%d" % (i, j, i+1, j+1))
    
print(H)

tensor([[0.2813, 0.3361, 0.5711],
        [0.3361, 0.6524, 0.9809],
        [0.5711, 0.9809, 2.0000]])


In [9]:
def lanczos(model, loss_function, batch, m):
    global H
    n = sum(p.data.numel() for p in model.parameters())
    v = np.ones(n) / np.sqrt(n)
    w = hessian_matmul(model, loss_function, torch.Tensor(v), batch).numpy()
    alpha = []
    alpha.append(w.dot(v))
    w -= alpha[0] * v
    
    V = [v]
    beta = []
    
    for i in range(1, m):
        b = np.linalg.norm(w)
        beta.append(b)
        if b > 0:
            v = w / b
        else:
            done = False
            k = 0
            while not done:
                k += 1
                v = np.random.rand(n)
                
                for v_ in V:
                    v -= v.dot(v_) * v_
                
                done = np.linalg.norm(n) > 0
                if k > n * 10:
                    raise Exception("Can't find orthogonal vector")
            
            v /= np.linalg.norm(n)
                
        for v_ in V:
            v -= v.dot(v_) * v_
            v /= np.linalg.norm(v)
               
        V.append(v)
        w = hessian_matmul(model, loss_function, torch.Tensor(v), batch).numpy()
        alpha.append(w.dot(v))
        w = w - alpha[-1] * V[-1] - beta[-1] * V[-2]

    T = np.diag(alpha)
    for i in range(m - 1):
        T[i, i + 1] = beta[i] 
        T[i + 1, i] = beta[i]

    V = np.array(V)
    return T, V

In [10]:
T, V = lanczos(model, loss_function, batch, m)

In [11]:
v = np.linalg.eig(T)[0]
v.sort()
v

array([0.08271125, 0.15347193, 2.69761154])

In [12]:
v = np.linalg.eig(H)[0]
v.sort()
v

array([0.08273784, 0.15341313, 2.697591  ], dtype=float32)

In [13]:
for i in range(m - 1):
    print("{:.3f}".format(V[i, :].dot(V[m - 1, :])))

0.000
0.000
